Some info about this notebook:
- Currently using resnet20 or resnet32 for cifar10, if you want to use other network, feel free to change it in the `train` function as that's where downloading the dataset is
- `depth` hyperparam in `general_hyperparameters` will allow toggle between resnet20 or 32
- Currently using `MultiStepLR` as learning rate scheduler
- Feel free to go through the `train` function as change as you see fit to add Apollo in!

In [1]:
from __future__ import print_function
import logging
import os
import sys
# sys.path.append("C:\\Users\\Tuan Tran\\Desktop\\CS595_final\\adahessian\\image_classification")

import numpy as np
import argparse
from tqdm import tqdm, tqdm_notebook, trange

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import datasets, transforms
from torch.autograd import Variable

from optim.adahessian_helpers.utils import *
from optim.adahessian_helpers.models.resnet import *
from optim.adahessian_helpers.optim_adahessian import Adahessian
from optim import Apollo
import matplotlib.pyplot as plt
import time

In [2]:
"""
Function to initialize optimizers, download data, lr scheduler then training loop and checkpoint best model
"""
def train(dataset_name,
          optimizer_str,
          lr,
          lr_decay,
          lr_decay_epoch,
          weight_decay,
          batch_size,
          test_batch_size,
          epochs,
          depth,
          seed):

    # set random seed to reproduce the work
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    # Create dir for checkpoint best model per optimizer
    if not os.path.isdir('checkpoint/'):
        os.makedirs('checkpoint/')
    
    print(f"Retrieving {dataset_name}")
    # get dataset
    train_loader, test_loader = getData(
        name=dataset_name, train_bs=batch_size, test_bs=test_batch_size)

    # make sure to use cudnn.benchmark for second backprop
    cudnn.benchmark = True

    # get model and optimizer
    model = resnet(num_classes=10, depth=depth).cuda()
    model = torch.nn.DataParallel(model)
    #print('    Total params: %.2fM' % (sum(p.numel()
    #                                       for p in model.parameters()) / 1000000.0))

    criterion = nn.CrossEntropyLoss()
    if optimizer_str == 'sgd':
        optimizer = optim.SGD(
            model.parameters(),
            lr=lr,
            momentum=0.9,
            weight_decay=weight_decay)
    elif optimizer_str == 'adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay)
    elif optimizer_str == 'adamw':
        print('For AdamW, we automatically correct the weight decay term for you! If this is not what you want, please modify the code!')
        weight_decay = weight_decay / lr
        optimizer = optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay)
    elif optimizer_str == 'adahessian':
        print('For AdaHessian, we use the decoupled weight decay as AdamW. Here we automatically correct this for you! If this is not what you want, please modify the code!')
        weight_decay = weight_decay / lr
        optimizer = Adahessian(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay)
    elif optimizer_str == "apollo":
        weight_decay = weight_decay / lr 
        optimizer = Apollo(
            model.parameters(),
            lr = lr,
            weight_decay = weight_decay,
            init_lr = lr / 5)
    else:
        raise Exception('We do not support this optimizer yet!!')

    # learning rate schedule
    scheduler = lr_scheduler.MultiStepLR(
        optimizer,
        lr_decay_epoch,
        gamma=lr_decay,
        last_epoch=-1)

    # Start training loop
    best_acc = 0.0
    losses_list = []
    acc_list = []
    
    for epoch in range(1, epochs + 1):
        print('Current Epoch: ', epoch)
        train_loss = 0.
        total_num = 0
        correct = 0

        scheduler.step()
        model.train()
        with tqdm_notebook(total=len(train_loader.dataset)) as progressbar:
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = data.cuda(), target.cuda()
                output = model(data)
                loss = criterion(output, target)
                loss.backward(create_graph=True)
                train_loss += loss.item() * target.size()[0]
                total_num += target.size()[0]
                _, predicted = output.max(1)
                correct += predicted.eq(target).sum().item()
                optimizer.step()
                optimizer.zero_grad()
                progressbar.update(target.size(0))

        acc = test(model, test_loader)
        train_loss /= total_num
        #print(f"Training Loss of Epoch {epoch}: {np.around(train_loss, 2)}")
        #print(f"Testing of Epoch {epoch}: {np.around(acc * 100, 2)}")
        losses_list.append(train_loss)
        acc_list.append(acc)
        
        if acc > best_acc:
            # Only display if found new best model
            print(f"Training Loss of Epoch {epoch}: {np.around(train_loss, 2)}")
            print(f"Testing of Epoch {epoch}: {np.around(acc * 100, 2)}")
            best_acc = acc
            torch.save({
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'best_accuracy': best_acc,
                }, f'checkpoint/{optimizer_str}_netbest.pkl')
    
    print(f'Best Acc for {optimizer_str} with dataset {dataset_name}: {np.around(best_acc * 100, 2)}')
    losses_plot_string = ("%s_%s_loss_plot.jpg" % (optimizer_str, dataset_name))
    accuracy_plot_string = ("%s_%s_accuracy_plot.jpg" % (optimizer_str, dataset_name))
    #losses plot
    loss_fig = plt.figure() 
    plt.plot(losses_list)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    loss_fig.suptitle("%s: Losses Vs Epoch" % (optimizer_str.upper()))
    plt.show()
    loss_fig.savefig(losses_plot_string)
    #accuracy plot
    acc_fig = plt.figure()
    plt.plot(acc_list)
    plt.ylabel("Accuracy")
    acc_fig.suptitle("%s: Accuracy Vs Epoch" % (optimizer_str.upper()))
    plt.show()
    acc_fig.savefig(accuracy_plot_string)

In [ ]:
"""
Training settings
"""
# Specific hyperparams for each optimizer
optim_to_params = {
    "sgd": {
        "lr": 0.1,
        'lr_decay': 0.1,
        'lr_decay_epoch': [80, 120],
        'weight_decay': 5e-4
    },
    "adam": {
        "lr": 0.001,
        'lr_decay': 0.1,
        'lr_decay_epoch': [80, 120],
        'weight_decay': 5e-4
    },
    "adamw": {
        "lr": 0.01,
        'lr_decay': 0.1,
        'lr_decay_epoch': [80, 120],
        'weight_decay': 5e-4
    },
    "adahessian": {
        "lr": 0.15,
        'lr_decay': 0.1,
        'lr_decay_epoch': [80, 120],
        'weight_decay': 5e-4
    },
    "apollo": {
        "lr": 0.375,
        'lr_decay': 0.1,
        'lr_decay_epoch': [80, 120],
        'weight_decay': 5e-4
    }    
}

# General hyperparameters that apply to all optimizers, feel free to change this around as you see fit
general_hyperparams = {
    'batch_size': 256,
    'test_batch_size': 256,
    'epochs': 5,
    # Depth of ResNet, 20 = resnet20, 32 = resnet32
    'depth': 32,
    'seed': 1
}

# cifar10 or mnist
dataset_name = "cifar10"

optimizers = ['apollo', 'adahessian']
# Run the loop to train using each optimizer in the list of optimizers
for optimizer_str in optimizers:
    print(f"{optimizer_str.upper()}")
    optim_hyperparams = optim_to_params[optimizer_str]
    train(dataset_name, optimizer_str, **optim_hyperparams, **general_hyperparams)

APOLLO
Retrieving cifar10
Files already downloaded and verified
Current Epoch:  1


/home/life/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/home/life/.local/lib/python3.6/site-packages/ipykernel_launcher.py:94: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
